## Question 3: Stefan-Botzmann constant
Note: nothing to hand in for part 5.12a according to the assignment sheet. 
### Part a
Note: this is 5.12b from Newman. 

The Planck theory of thermal radiation dictates that the amount of thermal energy per second electromagnetically radiated from a black body with unit area in an anglular frequency interval $\mathrm{d}\omega$ is the quantity $I(\omega)\mathrm{d}\omega$ where $I(\omega)$ represents the intensity of the thermal radiation as a function of angular frequency given as follows:
\begin{equation}
    I(\omega) = \frac{\hbar}{4\pi^2 c^2} \frac{\omega^3}{(e^{\hbar \omega/ k_B T} - 1)} \qquad (1)
\end{equation}
where $\hbar$, $k_B$, and $c$ are reduced Planck's constant ($h/(2\pi)$), Boltzmann's constant, and the speed of light in a vacuum, respectively. 

It can then be shown that the total rate of black body energy radiation per unit area over all frequencies, $W$, is:
\begin{equation}
    W = \frac{k_B^4 T^4}{4\pi^2 c^2 \hbar^3} \int_0^{\infty} \frac{x^3}{e^x - 1}\mathrm{d}x \qquad (2)
\end{equation}

Here, we set out to numerically approximate the integral given in equation (2) using the Trapezoid rule, and estimate the accuracy of this method of numerical integration in this case. We will use Newman's suggested method in section 5.2.1 for 'practical estimation of errors' for Trapezoid rule to comment on the accuracy of the result. That is, the error for evaluating an integral for $N_2$ steps is given by $\epsilon = \frac{1}{3}(I_2 - I_1)$ where $I_2$ is the value of the integral for $N_2$ steps and $I_1$ is the value of the integral for $N_1 = N_2/2$ steps. 

In order to perform this numerical integration we will need to perform a change of variables to acquire a finite upper bound of integration. This transformation is given in Newman as $x = \frac{z}{1-z}$. Simply substituting this change into equation (2) we see that the integrand in terms of $z$ becomes $\frac{z^3}{(e^{z/(1-z)}-1)(1-z)^5}$ and it is simple to see that the lower bound is still $0$ but the upper bound becomes $1$ when integrating with respect to $z$ since $z \rightarrow 1$ as $x \rightarrow \infty$.

Below is pseudocode highlighting the steps in this computation:

1. Import required packages.
2. Define the integrand as a function. NB: this should be the integrand resulting from the change of variables above. 
3. Define a general function for Trapezoid rule numerical integration with inputs: function to integrade, lower bound, upper bound, and number of steps. 
4. Define number of steps $N = 10$, a small quantity $\delta = 10^-15$, and bounds of integration $a = \delta$, $b = 1 - \delta$. This is effectively as close to integrating from $0$ to $1$ as we can get without incurring a dividing by zero error. 
5. Evaluate the integral by calling Trapezoid rule function evaluated for the integrand function, $N$ steps, $a$ LB, $b$ UB. 
6. Evaluate integral using Newman's practical estimation of errors method in 5.2.1 (see above).

In [2]:
#import numpy
import numpy as np

#define function to be integrated (after change of variables described in equation 5.67 Newman)
def Integrand(z):
    """
    This defines the mathematical funciton we want to integrate from 0 to 1 to approximate the total rate at which
    energy is radiated by a black bodt per unit area according to the Planck theory of thermal radiation
    INPUT:
    z [float] is the independent variable (after change of variables from x where the integral in terms of x was from 0 to inf)
    OUTPUT:
    res [float] is the image of input z under the integrand function.
    """
    res = z**3/((np.exp(z/(1-z))-1)*(1-z)**5)
    return res

#define a function for Trapezoid rule 
def Trapzzz(funktion, N, a, b):
    """
    A function for performing numerical integration of the mathematical function denoted funktion according to 
    the trapezoid rule for N steps.
    INPUT:
    funktion [function handle of single variable] is the function whose integral is to be numerically approximated 
    N [int] number of steps
    a [float] lower bound of integration
    b [float] upper bound of integration
    """
    h = (b-a)/N #width of trapezoids
    s = 0.5*funktion(a) + 0.5*funktion(b) #the constant terms in the series expansion defining the trapezoid rule as in equation 5.3 of Newman

    for j in range(1,N):
        s += funktion(a+j*h)
        
    the_integral = h*s
    return the_integral

N = 10 #let's try 10 steps
delta = 10**(-15) #<<1. if we integrate all the way to 1 we're going to get a division by 0 error due to the discretized nature of computation <--> BAD. But, we can get 'close'
a = delta #lower bound is 0 after change of variable. We shift it up by small delta so that (exp(z/(1-z))-1) term doesnt cause integrand to blow up
b = 1-delta #upper bound is 1 after change of variable. We shift it down by small delta so that the (1-z)^5 term doesnt cause integrand to blow up

theIntegral = Trapzzz(Integrand, N, a, b)
print('Numerical approximation via trapezoid rule of the Boltzmann black body power integral is: {0:.2f}'.format(theIntegral))

I2 = theIntegral
I1 = Trapzzz(Integrand, N//2, a, b)

err = (I2 - I1)/3

print('Practical error estimation method of error for Trapezoid rule integration yields an error estimate of {0:.2f}.'.format(err))
relative_err = err*100/I2
print('The estimated error is {0:.2f} % the magnitude of the integral approximation.'.format(relative_err))

Numerical approximation via trapezoid rule of the Boltzmann black body power integral is: 6.37
Practical error estimation method of error for Trapezoid rule integration yields an error estimate of -0.34.
The estimated error is -5.27 % the magnitude of the integral approximation.


The relatively small ratio ($\sim 5.27 \%$) between error estimate using Newman's practical error estimation method in comparison to magintidue of the integral approximation indicates that our result may be rather acccurate. We may represent the resulting integral approximation as $6.37 \pm 0.34$.

### Part b
Note: this is 5.12c from Newman. 

Recall from Part a that the total rate at which energy is radaited by a black body per unit area over all frequencies is given by equation (2). 

In part a we evaluated the integral from this expression $ \int_{0}^{\infty} \frac{x^3}{e^x -1} dx $ numerically using the Trapezoid rule with a change of variables to have finite bounds. 

As noted in Newman, Stefan's law also gives an expression for $W$ as follows:
\begin{equation}
    W = \sigma T^4 \qquad (3)
\end{equation}

Equation (2) and (3) ought to be equal for any given temperature $T$. We will use this equivalence and our numerical integration to estimate the Stefan-Boltzmann constant. More concretely, we will estimate $\sigma$ as follows:
\begin{equation}
    \sigma = \frac{k_B^4}{4 \pi^2 c^2 \hbar^3} \int_{0}^{\infty} \frac{x^3}{e^x -1} dx \qquad (4)
\end{equation}

Note that equation (4) is equal to $ W/T^4 $.

In [30]:
#define some constants 
k_b = 1.38 * 10**(-23) #J/K Boltzmann constant
c = 3*10**8 #m/s speed of light
hbar = 1.05 * 10 **(-34) #Js reduced Planck's constant

Stefan_est = k_b**4/(4*(np.pi**2)*(c**2)*hbar**3)*theIntegral
print('Estimation of Stefan-Botlzmann constant is: {0:.2E} W m^-2 K^-4.'.format(Stefan_est))

from scipy.constants import Stefan_Boltzmann

Stefan_acc = Stefan_Boltzmann
print('scipy.constant Stefan-Boltzmann value is {0:.2E} W m^-2 K^-4.'.format(Stefan_acc))

err = abs(Stefan_acc-Stefan_est)*100/Stefan_acc
print('Percent difference of our estimate of Stefan-Boltzmann constant from scipy value is {0:.2f} %.'.format(err))


Estimation of Stefan-Botlzmann constant is: 5.61E-08 W m^-2 K^-4.
scipy.constant Stefan-Boltzmann value is 5.67E-08 W m^-2 K^-4.
Percent difference of our estimate of Stefan-Boltzmann constant from scipy value is 1.00 %.


It is clear that even for a small number of steps, the Trapezoid rule yields a reasonable estimation of the integral in equation (2) given that we estimated the Stefan-Boltzmann constant to 3 significant figures with a percent difference as little as $1\%$. This reinforces our assertion in Part a that our integral approximation appears to be relatively accurate based off of the small resulting value in the error estimation. 